# 크롤링과 데이터베이스 

* cine21 배우 랭킹 사이트 크롤링
  - 사이트 주소: http://www.cine21.com/rank/person/
  - 요청 방식 확인 방법: 크롬 개발자 모드로 들어가서, Network -> content 페이지의 요청 방식 확인 (Preserve log, All 체크)
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form data (payload)
      - section = 'actor'
      - period_start = '2020-12'
      - gender = 'all'
      - page = 1 

### 1. 라이브러리 import

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

### 2. mongodb connection

In [2]:
conn = pymongo.MongoClient()
actor_db = conn.cine21 # cine21 데이터베이스 생성
actor_collection = actor_db.actor_collection # actor_collection 컬렉션 생성

### 3. 크롤링 주소 requests

In [3]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section']='actor'
post_data['period_start']='2020-12'
post_data['gender']='all'
post_data['page']=1

res = requests.post(cine21_url, data=post_data)

### 4. parsing과 배우 이름 추출

In [4]:
soup = BeautifulSoup(res.content, 'html.parser')

In [5]:
actors = soup.select('li.people_li div.name > a') # li는 리스트를 의미
for actor in actors:
    print(actor.text)

김윤석(6편)
조인성(2편)
허준호(2편)
김소진(4편)
조우진(7편)
황정민(5편)
정만식(4편)


In [6]:
test_data = '임지연(1편)'
re.sub('\(\w*\)','',test_data)

'임지연'

In [7]:
actors = soup.select('li.people_li div.name > a') # li는 리스트를 의미
for actor in actors:
    print(re.sub('\(\w*\)', '', actor.text))

김윤석
조인성
허준호
김소진
조우진
황정민
정만식


### 5. 배우 상세 정보 추출 

#### 참고: 특수한 정규 표현식
```html
  - Greedy(.*) vs Non-Greedy(.*?)
  - 연습: https://regexr.com
    - <li><span class="tit">직업</span>배우</li>
  - . 문자는 줄바꿈 문자인 \n 를 제외한 모든 문자 한개를 의미함
  - * 는 앞 문자가 0번 또는 그 이상 반복되는 패턴
  - .* 는 문자가 0번 또는 그 이상 반복되는 패턴
```

In [8]:
actors = soup.select('li.people_li div.name > a')
for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.attrs['href'] # 배우 링크
    response_actor = requests.get(actor_link) # 배우 링크 가져오기
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser') # 배우 링크 파싱하기
    default_info = soup_actor.select_one('ul.default_info') # 배우 기본 정보
    actor_details = default_info.select('li') # 기본 정보내에서 상세정보 데이터 추출
    for actor_item in actor_details: # 상세정보 출력
        print(actor_item) 
        print(actor_item.text)
        print(actor_item.select_one('span.tit').text) # 원하는 값
        print(re.sub('<span.*?>.*?</span>','',str(actor_item)))
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value)
        print(actor_item_value) # 원하는 값

<li><span class="tit">직업</span>배우</li>
직업배우
직업
<li>배우</li>
배우
<li><span class="tit">생년월일</span>1968-01-21</li>
생년월일1968-01-21
생년월일
<li>1968-01-21</li>
1968-01-21
<li><span class="tit">성별</span>남</li>
성별남
성별
<li>남</li>
남
<li><span class="tit">신장/체중</span>178cm, 68kg</li>
신장/체중178cm, 68kg
신장/체중
<li>178cm, 68kg</li>
178cm, 68kg
<li><span class="tit">학교</span>동의대학교 학사</li>
학교동의대학교 학사
학교
<li>동의대학교 학사</li>
동의대학교 학사
<li><span class="tit">취미</span>여행, 낚시</li>
취미여행, 낚시
취미
<li>여행, 낚시</li>
여행, 낚시
<li><span class="tit">소속사</span>심엔터테인먼트</li>
소속사심엔터테인먼트
소속사
<li>심엔터테인먼트</li>
심엔터테인먼트
<li><span class="tit">다른 이름</span>趙寅成</li>
다른 이름趙寅成
다른 이름
<li>趙寅成</li>
趙寅成
<li><span class="tit">직업</span>배우</li>
직업배우
직업
<li>배우</li>
배우
<li><span class="tit">생년월일</span>1981-07-28</li>
생년월일1981-07-28
생년월일
<li>1981-07-28</li>
1981-07-28
<li><span class="tit">성별</span>남</li>
성별남
성별
<li>남</li>
남
<li><span class="tit">신장/체중</span>186cm, 72kg</li>
신장/체중186cm, 72kg
신장/체중
<li>186cm, 72kg</li>
186cm, 72kg
<li><span class="tit">

In [9]:
actors = soup.select('li.people_li div.name > a')

actors_info_list = list()

for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.attrs['href'] # 배우 링크
    response_actor = requests.get(actor_link) # 배우 링크 가져오기
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser') # 배우 링크 파싱하기
    default_info = soup_actor.select_one('ul.default_info') # 배우 기본 정보
    actor_details = default_info.select('li') # 기본 정보내에서 상세정보 데이터 추출

    actor_info_dict = dict()
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text # 원하는 값
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value) # 원하는 값
        actor_info_dict[actor_item_field] = actor_item_value # 딕셔너리로 저장 'field':'value'
    actors_info_list.append(actor_info_dict)

actors_info_list        

[{'직업': '배우',
  '생년월일': '1968-01-21',
  '성별': '남',
  '신장/체중': '178cm, 68kg',
  '학교': '동의대학교 학사',
  '취미': '여행, 낚시',
  '소속사': '심엔터테인먼트'},
 {'다른 이름': '趙寅成',
  '직업': '배우',
  '생년월일': '1981-07-28',
  '성별': '남',
  '신장/체중': '186cm, 72kg',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '소속사': '싸이더스 HQ'},
 {'원어명': '許俊豪',
  '직업': '배우',
  '생년월일': '1964-03-03',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과',
  '취미': '만화책 보기',
  '특기': '야구, 농구',
  '소속사': '지티비엔터테인먼트'},
 {'직업': '배우', '성별': '여'},
 {'다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'},
 {'직업': '배우',
  '생년월일': '1970-09-01',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과 졸업',
  '특기': '농구, 악기연주',
  '소속사': '예당엔터테인먼트'},
 {'직업': '배우', '생년월일': '1974-12-11', '성별': '남'}]

### 6. 배우 흥행 지수와 출연 영화 추출

In [10]:
actors = soup.select('li.people_li div.name > a') 
hits = soup.select('ul.num_info > li > strong')
print(actors)
print(hits)

[<a href="/db/person/info/?person_id=57423">김윤석(6편)</a>, <a href="/db/person/info/?person_id=16259">조인성(2편)</a>, <a href="/db/person/info/?person_id=2425">허준호(2편)</a>, <a href="/db/person/info/?person_id=82573">김소진(4편)</a>, <a href="/db/person/info/?person_id=98611">조우진(7편)</a>, <a href="/db/person/info/?person_id=14268">황정민(5편)</a>, <a href="/db/person/info/?person_id=64972">정만식(4편)</a>]
[<strong>45,708</strong>, <strong>39,995</strong>, <strong>34,251</strong>, <strong>28,556</strong>, <strong>26,118</strong>, <strong>24,260</strong>, <strong>22,847</strong>]


In [11]:
actors = soup.select('li.people_li div.name > a') 
hits = soup.select('ul.num_info > li > strong')
for index,actor in enumerate(actors):
    print((re.sub('\(\w*\)', '', actor.text)))
    print(hits[index].text)

김윤석
45,708
조인성
39,995
허준호
34,251
김소진
28,556
조우진
26,118
황정민
24,260
정만식
22,847


In [12]:
actors = soup.select('li.people_li div.name > a') 
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')
for index,actor in enumerate(actors):
    print((re.sub('\(\w*\)', '', actor.text)))
    print(hits[index].text)
    movie_titles = movies[index].select('li a span') # 각 배우마다 영화출연 리스트가 있고 영화 별로 리스트로 나뉘어져 있음
    print(movie_titles)
    

김윤석
45,708
[<span>모가디슈</span>, <span>타짜</span>, <span>당신, 거기 있어줄래요</span>, <span>화이 : 괴물을 삼킨 아이</span>, <span>미성년</span>, <span>1987</span>]
조인성
39,995
[<span>모가디슈</span>, <span>클래식</span>]
허준호
34,251
[<span>모가디슈</span>, <span>국가부도의 날</span>]
김소진
28,556
[<span>모가디슈</span>, <span>스프링 송</span>, <span>아이 캔 스피크</span>, <span>미성년</span>]
조우진
26,118
[<span>발신제한</span>, <span>도굴</span>, <span>킹메이커</span>, <span>서복</span>, <span>국가부도의 날</span>, <span>봉오동 전투</span>]
황정민
24,260
[<span>인질</span>, <span>와이키키 브라더스</span>, <span>바람난 가족</span>, <span>신세계</span>, <span>내 생애 가장 아름다운 일주일</span>]
정만식
22,847
[<span>모가디슈</span>, <span>지푸라기라도 잡고 싶은 짐승들</span>, <span>똥파리</span>, <span>끝까지 간다</span>]


In [13]:
actors = soup.select('li.people_li div.name > a') 
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')
for index,actor in enumerate(actors):
    print((re.sub('\(\w*\)', '', actor.text)))
    print(hits[index].text)
    movie_titles = movies[index].select('li a span') 
    for movie_title in movie_titles:
        print(movie_title.text)

김윤석
45,708
모가디슈
타짜
당신, 거기 있어줄래요
화이 : 괴물을 삼킨 아이
미성년
1987
조인성
39,995
모가디슈
클래식
허준호
34,251
모가디슈
국가부도의 날
김소진
28,556
모가디슈
스프링 송
아이 캔 스피크
미성년
조우진
26,118
발신제한
도굴
킹메이커
서복
국가부도의 날
봉오동 전투
황정민
24,260
인질
와이키키 브라더스
바람난 가족
신세계
내 생애 가장 아름다운 일주일
정만식
22,847
모가디슈
지푸라기라도 잡고 싶은 짐승들
똥파리
끝까지 간다


In [14]:
actors = soup.select('li.people_li div.name > a') # 배우이름
hits = soup.select('ul.num_info > li > strong') # 흥행지수
movies = soup.select('ul.mov_list') # 출연영화 
for index, actor in enumerate(actors):
    print("배우이름:", (re.sub('\(\w*\)', '', actor.text)))
    print("흥행지수:", int(hits[index].text.replace(',','')))
    movie_titles = movies[index].select('li a span') # 각 배우마다 영화리스트가 있고 영화도 리스트로 나뉘어져 있음
    movie_title_list = list() # 리스트로 나뉘어진 영화를 하나의 리스트로 묶기 위함
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    print("출연영화:", movie_title_list)

배우이름: 김윤석
흥행지수: 45708
출연영화: ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']
배우이름: 조인성
흥행지수: 39995
출연영화: ['모가디슈', '클래식']
배우이름: 허준호
흥행지수: 34251
출연영화: ['모가디슈', '국가부도의 날']
배우이름: 김소진
흥행지수: 28556
출연영화: ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년']
배우이름: 조우진
흥행지수: 26118
출연영화: ['발신제한', '도굴', '킹메이커', '서복', '국가부도의 날', '봉오동 전투']
배우이름: 황정민
흥행지수: 24260
출연영화: ['인질', '와이키키 브라더스', '바람난 가족', '신세계', '내 생애 가장 아름다운 일주일']
배우이름: 정만식
흥행지수: 22847
출연영화: ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '똥파리', '끝까지 간다']


### 7. 수집한 데이터 기반 사전 만들기

In [15]:
# 4.배우이름 추출
actors = soup.select('li.people_li div.name > a') # li는 리스트를 의미
for actor in actors:
    print(re.sub('\(\w*\)', '', actor.text))

김윤석
조인성
허준호
김소진
조우진
황정민
정만식


In [16]:
# 5.배우 상세 정보 추출
actors = soup.select('li.people_li div.name > a')

actors_info_list = list()

for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.attrs['href'] # 배우 링크
    response_actor = requests.get(actor_link) # 배우 링크 가져오기
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser') # 배우 링크 파싱하기
    default_info = soup_actor.select_one('ul.default_info') # 배우 기본 정보
    actor_details = default_info.select('li') # 기본 정보내에서 상세정보 데이터 추출

    actor_info_dict = dict()
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text # 원하는 값
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value) # 원하는 값
        actor_info_dict[actor_item_field] = actor_item_value # 딕셔너리로 저장 'field':'value'
    actors_info_list.append(actor_info_dict)

actors_info_list        

[{'직업': '배우',
  '생년월일': '1968-01-21',
  '성별': '남',
  '신장/체중': '178cm, 68kg',
  '학교': '동의대학교 학사',
  '취미': '여행, 낚시',
  '소속사': '심엔터테인먼트'},
 {'다른 이름': '趙寅成',
  '직업': '배우',
  '생년월일': '1981-07-28',
  '성별': '남',
  '신장/체중': '186cm, 72kg',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '소속사': '싸이더스 HQ'},
 {'원어명': '許俊豪',
  '직업': '배우',
  '생년월일': '1964-03-03',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과',
  '취미': '만화책 보기',
  '특기': '야구, 농구',
  '소속사': '지티비엔터테인먼트'},
 {'직업': '배우', '성별': '여'},
 {'다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'},
 {'직업': '배우',
  '생년월일': '1970-09-01',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과 졸업',
  '특기': '농구, 악기연주',
  '소속사': '예당엔터테인먼트'},
 {'직업': '배우', '생년월일': '1974-12-11', '성별': '남'}]

In [17]:
# 6.배우 흥행 지수와 출연 영화 추출
actors = soup.select('li.people_li div.name > a') # 배우이름
hits = soup.select('ul.num_info > li > strong') # 흥행지수
movies = soup.select('ul.mov_list') # 출연영화 
for index, actor in enumerate(actors):
    print("배우이름:", (re.sub('\(\w*\)', '', actor.text)))
    print("흥행지수:", int(hits[index].text.replace(',','')))
    movie_titles = movies[index].select('li a span') # 각 배우마다 영화리스트가 있고 영화도 리스트로 나뉘어져 있음
    movie_title_list = list() # 리스트로 나뉘어진 영화를 하나의 리스트로 묶기 위함
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    print("출연영화:", movie_title_list)

배우이름: 김윤석
흥행지수: 45708
출연영화: ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']
배우이름: 조인성
흥행지수: 39995
출연영화: ['모가디슈', '클래식']
배우이름: 허준호
흥행지수: 34251
출연영화: ['모가디슈', '국가부도의 날']
배우이름: 김소진
흥행지수: 28556
출연영화: ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년']
배우이름: 조우진
흥행지수: 26118
출연영화: ['발신제한', '도굴', '킹메이커', '서복', '국가부도의 날', '봉오동 전투']
배우이름: 황정민
흥행지수: 24260
출연영화: ['인질', '와이키키 브라더스', '바람난 가족', '신세계', '내 생애 가장 아름다운 일주일']
배우이름: 정만식
흥행지수: 22847
출연영화: ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '똥파리', '끝까지 간다']


In [18]:
actors_info_list = list()

actors = soup.select('li.people_li div.name > a') # 배우이름
hits = soup.select('ul.num_info > li > strong') # 흥행지수
movies = soup.select('ul.mov_list') # 출연영화 
rankings = soup.select('li.people_li > span.grade') # 랭킹

for index, actor in enumerate(actors):
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_hits = int(hits[index].text.replace(',',''))
    movie_titles = movies[index].select('li a span')
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    actor_info_dict = dict()
    actor_info_dict['배우이름'] = actor_name
    actor_info_dict['흥행지수'] = actor_hits
    actor_info_dict['출연영화'] = movie_title_list
    actor_info_dict['랭킹'] = rankings[index].text
    
    actor_link = 'http://www.cine21.com' + actor.attrs['href'] # 배우 링크
    response_actor = requests.get(actor_link) # 배우 링크 가져오기
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser') # 배우 링크 파싱하기
    default_info = soup_actor.select_one('ul.default_info') # 배우 기본 정보
    actor_details = default_info.select('li') # 기본 정보내에서 상세정보 데이터 추출
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text # 원하는 값
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value) # 원하는 값
        actor_info_dict[actor_item_field] = actor_item_value # 딕셔너리로 저장 'field':'value'
    actors_info_list.append(actor_info_dict)

actors_info_list
    

[{'배우이름': '김윤석',
  '흥행지수': 45708,
  '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'],
  '랭킹': '1',
  '직업': '배우',
  '생년월일': '1968-01-21',
  '성별': '남',
  '신장/체중': '178cm, 68kg',
  '학교': '동의대학교 학사',
  '취미': '여행, 낚시',
  '소속사': '심엔터테인먼트'},
 {'배우이름': '조인성',
  '흥행지수': 39995,
  '출연영화': ['모가디슈', '클래식'],
  '랭킹': '2',
  '다른 이름': '趙寅成',
  '직업': '배우',
  '생년월일': '1981-07-28',
  '성별': '남',
  '신장/체중': '186cm, 72kg',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '소속사': '싸이더스 HQ'},
 {'배우이름': '허준호',
  '흥행지수': 34251,
  '출연영화': ['모가디슈', '국가부도의 날'],
  '랭킹': '3',
  '원어명': '許俊豪',
  '직업': '배우',
  '생년월일': '1964-03-03',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과',
  '취미': '만화책 보기',
  '특기': '야구, 농구',
  '소속사': '지티비엔터테인먼트'},
 {'배우이름': '김소진',
  '흥행지수': 28556,
  '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년'],
  '랭킹': '4',
  '직업': '배우',
  '성별': '여'},
 {'배우이름': '조우진',
  '흥행지수': 26118,
  '출연영화': ['발신제한', '도굴', '킹메이커', '서복', '국가부도의 날', 

### 8. 여러 페이지의 배우 상세 정보 추출하기

In [19]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section']='actor'
post_data['period_start']='2020-12'
post_data['gender']='all'

for index in range(1,21):
    post_data['page'] = index
    
    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    actors = soup.select('li.people_li div.name > a') # 배우이름
    hits = soup.select('ul.num_info > li > strong') # 흥행지수
    movies = soup.select('ul.mov_list') # 출연영화 
    rankings = soup.select('li.people_li > span.grade') # 랭킹

    for index, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_hits = int(hits[index].text.replace(',',''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)

        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = rankings[index].text

        actor_link = 'http://www.cine21.com' + actor.attrs['href'] # 배우 링크
        response_actor = requests.get(actor_link) # 배우 링크 가져오기
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser') # 배우 링크 파싱하기
        default_info = soup_actor.select_one('ul.default_info') # 배우 기본 정보
        actor_details = default_info.select('li') # 기본 정보내에서 상세정보 데이터 추출

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text # 원하는 값
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value) # 원하는 값
            actor_info_dict[actor_item_field] = actor_item_value # 딕셔너리로 저장 'field':'value'
        actors_info_list.append(actor_info_dict)

print(actors_info_list)

[{'배우이름': '김윤석', '흥행지수': 45708, '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'], '랭킹': '1', '직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}, {'배우이름': '조인성', '흥행지수': 39995, '출연영화': ['모가디슈', '클래식'], '랭킹': '2', '다른 이름': '趙寅成', '직업': '배우', '생년월일': '1981-07-28', '성별': '남', '신장/체중': '186cm, 72kg', '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)', '취미': '모자모으기, 영화보기, 농구', '특기': '농구, 태권도(공인 4단)', '소속사': '싸이더스 HQ'}, {'배우이름': '허준호', '흥행지수': 34251, '출연영화': ['모가디슈', '국가부도의 날'], '랭킹': '3', '원어명': '許俊豪', '직업': '배우', '생년월일': '1964-03-03', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과', '취미': '만화책 보기', '특기': '야구, 농구', '소속사': '지티비엔터테인먼트'}, {'배우이름': '김소진', '흥행지수': 28556, '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년'], '랭킹': '4', '직업': '배우', '성별': '여'}, {'배우이름': '조우진', '흥행지수': 26118, '출연영화': ['발신제한', '도굴', '킹메이커', '서복', '국가부도의 날', '봉오동 전투'], '랭킹': '5', '다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}, {'

### 9. mongodb에 크롤링 데이터 저장하기

In [ ]:
conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actor_collection.insert_many(actors_info_list)

In [ ]:
actor_collection.delete_many({})

### 10. 배우 랭킹 및 상세 정보 클롤링 및 mongodb 저장

In [ ]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section']='actor'
post_data['period_start']='2020-12'
post_data['gender']='all'

for index in range(1,21):
    post_data['page'] = index
    
    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    actors = soup.select('li.people_li div.name > a') # 배우이름
    hits = soup.select('ul.num_info > li > strong') # 흥행지수
    movies = soup.select('ul.mov_list') # 출연영화 
    rankings = soup.select('li.people_li > span.grade') # 랭킹

    for index, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_hits = int(hits[index].text.replace(',',''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)

        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = rankings[index].text

        actor_link = 'http://www.cine21.com' + actor.attrs['href'] # 배우 링크
        response_actor = requests.get(actor_link) # 배우 링크 가져오기
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser') # 배우 링크 파싱하기
        default_info = soup_actor.select_one('ul.default_info') # 배우 기본 정보
        actor_details = default_info.select('li') # 기본 정보내에서 상세정보 데이터 추출

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text # 원하는 값
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value) # 원하는 값
            actor_info_dict[actor_item_field] = actor_item_value # 딕셔너리로 저장 'field':'value'
        actors_info_list.append(actor_info_dict)
        
actor_collection.insert_many(actors_info_list)